# Build an Agent

- 에이전트(agent)는 어떤 행동을하고 입력값을 결정하기 위해 LLM을 추론 엔진(reasoning engine)으로 사용한다.
- 행동을 한 후에는 추가적인 행동이 필요한지 아니면 종료해도 되는지 결정하기 위해 결과를 LLM에게 입력한다.

# End-to-end agent

- 아래 코드 스니펫은 LLM이 어느 도구를 사용할지 결정하는 완전한 함수형 에이전트를 보여준다.

In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {
        "messages": [HumanMessage(content="hi im bob! and i live in sf")]
    },
    config=config
):
  print(chunk)
  print("----")

for chunk in agent_executor.stream(
    {
        "messages": [HumanMessage(content="whats the weather where i live?")]
    },
    config=config
):
  print(chunk)
  print("----")

# Setup

In [ ]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python

In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


# Define tools

- Tavily 검색 엔진을 도구로 쉽게 사용하기 위해 LangChain의 내장 도구를 사용한다.

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")

print(search_results)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later

tools = [search]

[{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/94161/date/2024-6-21', 'content': 'Current Weather for Popular Cities . San Francisco, CA warning 56 ° F Fair; Manhattan, NY warning 73 ° F Sunny; Schiller Park, IL (60176) warning 78 ° F Partly Cloudy; Boston, MA warning 73 ...'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718943994, 'localtime': '2024-06-20 21:26'}, 'current': {'last_updated_epoch': 1718943300, 'last_updated': '2024-06-20 21:15', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 9.4, 'wind_kph': 15.1, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 84, 'cloud': 100, 'f

# Using Language Models

In [6]:
pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 2.8 MB/s eta 0:00:00


In [7]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

··········


In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

- 언어 모델의 검색 엔진 도구 호출을 활성하시키기 위해서 bind_tools 메소드를 사용한다.

In [9]:
model_with_tools = model.bind_tools(tools)

- 이제 모델을 호출해보자. 먼저 일반 메세지로 모델을 호출하고 어떻게 응답이 오는지 확인한다.

In [10]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


- 이번에는 도구가 호출되도록 메세지를 수정한다.

In [11]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_zIxTfipWYe9f8TPPgAddtlj5'}]


- 응답의 content 값이 존재하지 않지만 tool_calls 값은 존재한다.

# Create the agent

- 에이전트를 생성하기 위해 LangGraph를 사용할 것이다.
- LLM과 도구를 가지고 에이전트를 초기화할 수 있다.

In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the agent

- 이제 몇 가지 쿼리로 에이전트를 실행할 수 있다.

In [13]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='f4570329-a3aa-40e6-90d3-f242c583fe2d'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b504ac44-f219-41dd-b016-faf7a3b04552-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [14]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)

response["messages"]

[HumanMessage(content='whats the weather in sf?', id='c0ce2b1f-8647-4015-9ee2-4e59b071b63d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6zqaNuRl2fOfU8RHpCsKV3hD', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e35904d9-d2b7-406e-b64a-f5ebf8d96040-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_6zqaNuRl2fOfU8RHpCsKV3hD'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109}),
 ToolMessage(content='[{"url": "https://world-weather.info/forecast/usa/san_francisco/june-2024/", "content": "Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Deta

# Streaming Messages

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gRFOgcYAhkk8IGtqvJzct25A', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-248d640c-2ea4-4e3c-921a-7d88907f44bd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_gRFOgcYAhkk8IGtqvJzct25A'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://world-weather.info/forecast/usa/san_francisco/june-2024/", "content": "Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed \\u26a1 San Francisco Weather Foreca

# Streaming tokens

In [18]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.wunderground.com/hourly/us/ca/san-francisco/94161/date/2024-6-21', 'content': 'San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718950020, 'localtime': '2024-06-20 23:07'}, 'current': {'last_updated_epoch': 1718949600, 'last_updated': '2024-06-20 23:00', 'temp_c': 13.3, 'temp_f': 55.9, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 1

# Adding in memory

- 앞서 언급한 대로, 에이전트는 무상태이다. 즉, 에이전트는 과거의 상호작요을 기억하지 못한다.
- 에이전트에게 메모리 기능을 추가하기 위해서 체크 포인터를 전달해줄 필요가 있다.
- 에이전트를 실행할 때 체크포인터와 함께 thread_id도 전달해줘야 한다.

In [19]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cce149da-de39-4b87-8bfd-565ee34c9862-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 108, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6b9384f9-63ce-4c43-99c2-985f1923134a-0', usage_metadata={'input_tokens': 108, 'output_tokens': 14, 'total_tokens': 122})]}}
----


- 만약 새로운 대화를 시작하고 싶다면, thread_id만 변경해주면 된다.

In [23]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
  print(chunk)
  print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal information like your name. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 86, 'total_tokens': 106}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-333c4831-440d-4ae5-912c-336b8a5d8777-0', usage_metadata={'input_tokens': 86, 'output_tokens': 20, 'total_tokens': 106})]}}
----
